In [1]:
%cd ../

/u02/thanh/workplace/plasma


In [2]:
from packages.package1 import CONTEXT

In [3]:
CONTEXT

|->class4
  |->class1
    |->x1
    |->y1:int
  |->class2
    |->class1...
    |->class3
      |->x3
      |->y3=int
    |->x2:int|str
    |->y2
  |->class3...
  |->class6 --> class6:packages.package2
|->factory:Factory
  |->module1
  |->module2
|->class7
  |->x1
  |->y1:int
|->class5
  |->class1...
  |->class3...

In [4]:
class1, class7 = CONTEXT.run('class1', 'class7', x1=8, y1=9)

In [5]:
class1, class7

(<packages.package1.class1.Class1 at 0x797db54deba0>,
 <packages.package1.class1.Class1 at 0x797db54deba0>)

In [7]:
class1 is class7

True